In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
os.chdir('/content/drive/MyDrive/015GithubRepos/da_for_sbr')

In [3]:
# from google.colab import drive
# drive.mount('/content/drive')

# import os
# os.chdir('/content/drive/MyDrive/015GithubRepos/da_for_sbr')

        

import pickle
import os
import matplotlib.pyplot as plt
from collections import Counter

from preprocessing import *
from simmetric import *

experiment = 1

dataPATH = '/content/drive/MyDrive/015GithubRepos/da_for_sbr/exps'
expPATH = '/content/drive/MyDrive/015GithubRepos/da_for_sbr/exps'


# dataPATH = 'G:/내 드라이브/015GithubRepos/da_for_sbr/exps'
# expPATH = 'G:/내 드라이브/015GithubRepos/da_for_sbr/exps'


if not os.path.exists(f'exps/experiment{experiment}'):
    os.makedirs(f'exps/experiment{experiment}')

In [ ]:
#%% 폴더 만들고 저장

if not os.path.exists(f'exps/experiment{experiment}/yoochoose'):
    os.makedirs(f'exps/experiment{experiment}/yoochoose')
    
if not os.path.exists(f'exps/experiment{experiment}/diginetica'):
    os.makedirs(f'exps/experiment{experiment}/diginetica')
    
if not os.path.exists(f'exps/experiment{experiment}/result_narm_yoochoose'):
    os.makedirs(f'exps/experiment{experiment}/result_narm_yoochoose')
    
if not os.path.exists(f'exps/experiment{experiment}/result_narm_diginetica'):
    os.makedirs(f'exps/experiment{experiment}/result_narm_diginetica')
    
if not os.path.exists(f'exps/experiment{experiment}/result_srgnn_yoochoose'):
    os.makedirs(f'exps/experiment{experiment}/result_srgnn_yoochoose')
    
if not os.path.exists(f'exps/experiment{experiment}/result_srgnn_diginetica'):
    os.makedirs(f'exps/experiment{experiment}/result_srgnn_diginetica')



# 원본데이터 불러오기, 전처리

In [5]:
y_or_d = 'yoochoose'  # yoochoose or diginetica

In [ ]:
#%% 한 번만 실행하면됨

y_or_d = 'diginetica'

if y_or_d == 'yoochoose':
    filename = 'yoochoose-clicks-withHeader.dat'  # 원본 데이터 파일
    tra_sess, tes_sess, sess_clicks = prep_yoochoose(dataPATH, filename)
    fracs = [1/64, 1/128, 1/256, 1/512]
    
else:
    filename = 'train-item-views.csv'  # 원본 데이터 파일
    tra_sess, tes_sess, sess_clicks = prep_diginetica(dataPATH, filename)
    fracs = [1/1, 1/3, 1/6, 1/12]

# 원본데이터 전처리(통계량 확인차)
print(f'원본데이터 통계량')
tra_ids_ori, tra_dates_ori, tra_seqs_ori, item_dict_ori = obtian_tra(sess_clicks, tra_sess)

0it [00:00, ?it/s]

원래 데이터의 training 세션 수 : 186670
-- Starting @ %ss : 2021-12-24 15:25:09.154260+09:00
약 26초


1235380it [00:14, 87088.01it/s]


-- Reading data @ 2021-12-24 15:25:24.046016s
Splitting date 1464102000.0
원본데이터 통계량
train 세션 수 : 186670
train 아이템 수 :  43097


In [ ]:
#%% train 데이터로 저장 (마지막 아이템 떼서 라벨로)
from preprocessing import *
for f in fracs:
    split_frac = int(len(tra_sess) * f)
    tra_sess_frac = tra_sess[-split_frac:]
    
    print(f'----------------------------------------------')
    print(f'frac : 1/{int(1/f)}')
    tra_ids_frac, tra_dates_frac, tra_seqs_frac, item_dict_frac = obtian_tra(sess_clicks, tra_sess_frac)
    tes_ids, tes_dates, tes_seqs = obtian_tes(sess_clicks, tes_sess, item_dict_frac)
    save_tra_seqs_frac(experiment, y_or_d, f, tra_seqs_frac)

    # frac train데이터 저장
    tr_seqs_frac, tr_dates_frac, tr_labs_frac, tr_ids_frac = process_seqs(tra_seqs_frac, tra_dates_frac)
    te_seqs, te_dates, te_labs, te_ids = process_seqs(tes_seqs, tes_dates)
    tra_frac = (tr_seqs_frac, tr_labs_frac)
    tes = (te_seqs, te_labs)
    
    train_name = f'exps/experiment{experiment}/{y_or_d}/{y_or_d[0]}{int(1/f):03}_train.txt'
    print(f'train_name : {train_name}')
    test_name = f'exps/experiment{experiment}/{y_or_d}/{y_or_d[0]}{int(1/f):03}_test.txt'
    print(f'test_name : {test_name}')
    pickle.dump(tra_frac, open(train_name, 'wb'))
    pickle.dump(tes, open(test_name, 'wb'))

----------------------------------------------
frac : 1/1
train 세션 수 : 186670
train 아이템 수 :  43097
test 세션 총 개수 : 15963
test 데이터 아이템 수 :  21131
tra_seqs save dir : exps/experiment1/diginetica/d001_tra_seqs.pkl
train_name : exps/experiment1/diginetica/d001_train.txt
test_name : exps/experiment1/diginetica/d001_test.txt
----------------------------------------------
frac : 1/3
train 세션 수 : 62223
train 아이템 수 :  36427
test 세션 총 개수 : 15644
test 데이터 아이템 수 :  19651
tra_seqs save dir : exps/experiment1/diginetica/d003_tra_seqs.pkl
train_name : exps/experiment1/diginetica/d003_train.txt
test_name : exps/experiment1/diginetica/d003_test.txt
----------------------------------------------
frac : 1/6
train 세션 수 : 31111
train 아이템 수 :  29129
test 세션 총 개수 : 15000
test 데이터 아이템 수 :  17063
tra_seqs save dir : exps/experiment1/diginetica/d006_tra_seqs.pkl
train_name : exps/experiment1/diginetica/d006_train.txt
test_name : exps/experiment1/diginetica/d006_test.txt
------------------------------------------

In [6]:
from preprocessing import *

frac = 1/64

# tra_seqs_frac 불러오기
tra_seqs_frac = load_tra_seqs_frac(experiment, y_or_d, frac)

# 아이템의 출현 횟수 확인하기
allsess = []
for s in tra_seqs_frac:
  allsess += s

allaprcnt = len(allsess)
print(f'모든 세션의 아이템 출현 수 : {allaprcnt}')

# 아이템별 출현 횟수 카운트
allitemcntr = Counter(allsess)

# 아이템의 개수 구하기
nof_items = len(allitemcntr)

print(f'총 아이템 수 : {nof_items}')

loading file : exps/experiment1/yoochoose/y064_tra_seqs.pkl
124472
모든 세션의 아이템 출현 수 : 519275
총 아이템 수 : 17597


In [ ]:
from simmetric import *

co_mat_win = get_co_matrix_win(tra_seqs_frac, 5, True, True)
save_mat_as_csr(co_mat_win, 'coo_mat', experiment, y_or_d, frac)

pmi_mat = ppmi(co_mat_win, True)
save_mat_as_csr(pmi_mat, 'pmi_mat', experiment, y_or_d, frac)

jac_mat = get_jaccard_mat(tra_seqs_frac, 5, diag_freq = True, remove_dup = True)
save_mat_as_csr(jac_mat, 'jac_mat', experiment, y_or_d, frac)

tan_mat = get_tanimoto(tra_seqs_frac, window_size=5, diag_freq=True, remove_dup=True)
save_mat_as_csr(tan_mat, 'tan_mat', experiment, y_or_d, frac)

cos_mat_win = get_cosine(co_mat_win)
save_mat_as_csr(cos_mat_win, 'cos_mat', experiment, y_or_d, frac)

w2v_mat = get_w2v_model(tra_seqs_frac, nof_items)
save_mat_as_csr(w2v_mat, 'w2v_mat', experiment, y_or_d, frac)

저장파일 : exps/experiment1/yoochoose/y064_csr_coo_mat.pkl


/content/drive/MyDrive/015GithubRepos/da_for_sbr/simmetric.py:104: RuntimeWarning: invalid value encountered in long_scalars
  pmi = np.log2(C[i, j] * N / (S[j]*S[i]) + eps)  # np.log2(0)이 음의 무한대가 되는 것을 막기위해 eps 사용함


0.0 % 완료
17.047391749062392 % 완료
34.094783498124784 % 완료
51.14217524718718 % 완료
68.18956699624957 % 완료
85.23695874531197 % 완료
저장파일 : exps/experiment1/yoochoose/y064_csr_pmi_mat.pkl
0.0 % 완료
5.682463916354131 % 완료
11.364927832708261 % 완료
17.047391749062392 % 완료
22.729855665416522 % 완료
28.412319581770657 % 완료
34.094783498124784 % 완료
39.777247414478914 % 완료
45.459711330833045 % 완료
51.14217524718718 % 완료
56.82463916354131 % 완료
62.507103079895444 % 완료
68.18956699624957 % 완료
73.8720309126037 % 완료
79.55449482895783 % 완료
85.23695874531197 % 완료
90.91942266166609 % 완료
96.60188657802023 % 완료
저장파일 : exps/experiment1/yoochoose/y064_csr_jac_mat.pkl
10.00% 완료
20.00% 완료
30.00% 완료
40.00% 완료
50.00% 완료
60.00% 완료
70.00% 완료
80.00% 완료
90.00% 완료
100.00% 완료
저장파일 : exps/experiment1/yoochoose/y064_csr_tan_mat.pkl
10.00% 완료
20.00% 완료
